In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import os
import matplotlib.pyplot as plt
import random

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)

set_all_seeds(420)

In [ ]:
model = MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)

In [ ]:
model.summary()

In [ ]:
!mkdir gdrive/MyDrive/flower_data/encoded

In [ ]:
!mkdir gdrive/MyDrive/flower_data/encoded/test gdrive/MyDrive/flower_data/encoded/train gdrive/MyDrive/flower_data/encoded/val

In [ ]:
layer_name = 'block_15_add'
feature_extractor = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

In [ ]:
for set_kind in ["train", "val", "test"]:
    source_dir = "gdrive/MyDrive/flower_data/augmented_new/" + set_kind
    sink_dir = "gdrive/MyDrive/flower_data/encoded/" + set_kind

    for i, directory in enumerate(os.listdir(source_dir)):
        print(f"\r[{i}] {source_dir}/{directory}", end='')
        dump = []

        for i, file in enumerate(os.listdir(f"{source_dir}/{directory}")):
            image = tf.keras.preprocessing.image.load_img(f"{source_dir}/{directory}/{file}")
            x = tf.keras.preprocessing.image.img_to_array(image)
            dump += [x]

        preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(np.array(dump))
        results = feature_extractor.predict(preprocessed)
        np.save(f"{sink_dir}/{directory}.npy", results)

In [ ]:


# Przygotowanie modelu do ekstrakcji cech
feature_extractor = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

for set_kind in ["train", "val", "test"]:
    source_dir = "gdrive/MyDrive/flower_data/augmented_new/" + set_kind
    sink_dir = "gdrive/MyDrive/flower_data/encoded/" + set_kind

    for i, directory in enumerate(os.listdir(source_dir)):
        print(f"\r[{i}] {source_dir}/{directory}", end='')
        dump = []

        for i, file in enumerate(os.listdir(f"{source_dir}/{directory}")):
            image = tf.keras.preprocessing.image.load_img(f"{source_dir}/{directory}/{file}")
            x = tf.keras.preprocessing.image.img_to_array(image)
            dump.append(x)

        preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(np.array(dump))
        results = feature_extractor.predict(preprocessed)
        np.save(f"{sink_dir}/{directory}.npy", results)
